# FLSim Tutorial: Features to compress client updates for communication efficiency


## Introduction

In this tutorial, we will look at how to use FL Simulator channels to compress client to server messages for communication efficiency. Several approaches, such as quantization, sparsity and sketching have been shown to be useful, and here we guide you through using support in the simulator for three basic techniques, namely half-precision, scalar quantization and Top-K sparsity.

### Prerequisites

Communication efficiency in federated learning is an important problem, as we have to transmit messages between the server (which is usually in a datacenter) and clients, which for example could be mobile devices communicating through a low-bandwidth channel. This causes a bottleneck, specially for the download phase where clients upload model updates to the server. One of the ways this can be addressed is by applying different techniques for compressing the client update. To get started with the motivation for this and get introduced to some of the background techniques, here is a recommended reading list:
- Jakub Konečný, H. Brendan McMahan, Felix X. Yu, Peter Richtarik, Ananda Theertha Suresh, Dave Bacon, "Federated Learning: Strategies for Improving Communication Efficiency", 
NIPS Workshop on Private Multi-Party Machine Learning (2016) https://research.google/pubs/pub45648/
- Sattler, Felix, et al. "Robust and communication-efficient federated learning from non-iid data." IEEE transactions on neural networks and learning systems 31.9 (2019): 3400-3413. https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8889996


Now that you're familiar with the motivation and basic techniques for communication efficiency, let's get started!


### Objectives 

By the end of this tutorial, we will have learnt how to

1. Setup a basic model training pipeline on the Cifar10 task
2. Configure different channels for quantization and sparsity
3. Setup channel measurements for tracking communication
5. Launch FL training flows using FLSim with quantization and sparsity

## Setup a basic training pipeline for the Cifar10 task

### Prerequisite
Let's install flsim via pip with the command below.

In [38]:
!pip install --quiet flsim

Subsequently, let us define the data pipeline and model for training on the Cifar10 task. This is similar to what we have done in the Cifar-10 tutorial at https://github.com/facebookresearch/FLSim/blob/main/tutorials/cifar10_tutorial.ipynb. 

In [39]:
USE_CUDA = True
LOCAL_BATCH_SIZE = 32
EXAMPLES_PER_USER = 500
IMAGE_SIZE = 32

We set up the data pipeline for this task and import the model we will be training on Cifar-10. Similar to the Cifar-10 tutorial, it will be a standard four-layer convolutional neural network, with group norms and a linear layer. We also import the metrics reporter as in the original tutorial.

In [40]:
import torch
from torchvision.datasets.cifar import CIFAR10
from flsim.utils.example_utils import FLModel
from flsim.utils.example_utils import SimpleConvNet

cuda_enabled = torch.cuda.is_available() and USE_CUDA
device = torch.device(f"cuda:{0}" if cuda_enabled else "cpu")

We define a config for training on the Cifar-10 dataset with federated learning. This does not incorporate any settings for client update compression yet, as we will be showing in subsequent sections how to choose a technique and define the configuration for it. We will be modifying this config to incorporate different approaches such as half-precision, scalar quantization and sparsity, and also guide you through their compression parameters.

In [41]:
import flsim.configs
from flsim.utils.config_utils import fl_config_from_json
from omegaconf import OmegaConf


json_config = {
    "trainer": {
        "_base_": "base_sync_trainer",
        # there are different types of aggregators
        # fed avg doesn't require lr, while others such as fed_avg_with_lr or fed_adam do
        "_base_": "base_sync_trainer",
        "server": {
          "_base_": "base_sync_server",
          "server_optimizer": {
            "_base_": "base_fed_avg_with_lr",
            "lr": 2.13,
            "momentum": 0.9
          },
          # type of user selection sampling
          "active_user_selector": {"_base_": "base_uniformly_random_active_user_selector"},
        },
        "client": {
            # number of client's local epoch
            "epochs": 1,
            "optimizer": {
                "_base_": "base_optimizer_sgd",
                # client's local learning rate
                "lr": 0.0138,
                # client's local momentum
                "momentum": 0,
            },
        },
        # number of users per round for aggregation
        "users_per_round": 5,
        # total number of global epochs
        # total #rounds = ceil(total_users / users_per_round) * epochs
        "epochs": 1,
        # frequency of reporting train metrics
        "train_metrics_reported_per_epoch": 100,
        # frequency of evaluation per epoch
        "eval_epoch_frequency": 1,
        "do_eval": True,
        # should we report train metrics after global aggregation
        "report_train_metrics_after_aggregation": True,
    }
}

We now define a function *run_cifar10_fl_training()* which runs a round of FL training, and takes as input the FL configuration. We can train under different compression techniques by appropriately changing the config as needed.

In [42]:
from flsim.utils.example_utils import build_data_provider, MetricsReporter

data_provider = build_data_provider(
    local_batch_size=LOCAL_BATCH_SIZE,
    examples_per_user=EXAMPLES_PER_USER,
    image_size = IMAGE_SIZE,
)

from flsim.interfaces.metrics_reporter import Channel

from hydra.utils import instantiate

def run_cifar10_fl_training(json_config):
  model = SimpleConvNet(in_channels=3, num_classes=10)
  global_model = FLModel(model, device)

  if cuda_enabled:
      global_model.fl_cuda()

  cfg = fl_config_from_json(json_config)
  trainer_config = cfg.trainer
  trainer = instantiate(trainer_config, model=global_model, cuda_enabled=cuda_enabled)

  metrics_reporter = MetricsReporter([Channel.TENSORBOARD, Channel.STDOUT])

  final_model, eval_score = trainer.train(
      data_provider=data_provider,
      metrics_reporter=metrics_reporter,
      num_total_users=data_provider.num_train_users(),
      distributed_world_size=1
  )

Files already downloaded and verified
Files already downloaded and verified


Creating FL User: 100user [00:24,  4.11user/s]
Creating FL User: 20user [00:03,  5.33user/s]
Creating FL User: 20user [00:03,  5.37user/s]

Clients in total: 100


Let us first train a vanilla setting where there is no client update compression, and no metrics reporting. 

In [43]:
run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.312931588292122
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 13.88
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:26,  1.42s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.1286049485206604
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 27.08
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.1240725725889207
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 24.12
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:03<00:27,  1.53s/round]

...

Round:  90%|█████████ | 18/20 [00:22<00:02,  1.07s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.3562731005251407
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 51.28
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.2776811376214028
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 54.84
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:23<00:01,  1.07s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.2884995967149735
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 53.2
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.2883846752345562
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 53.64
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.320712050050497
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 52.08
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, global round = 20), Loss/Eval: 1.3698589166626334
(epoch = 1, round = 20, global round = 20), Accuracy/Eval: 51.25


Epoch:   0%|          | 0/1 [00:25<?, ?epoch/s]


## Introduction to communication metrics reporting in FL Simulator

We support tracking of communication metrics through the flag *report_communication_metrics*, and for every round report the mean number of bytes transmitted both for uploads (client to server communication), as well as downloads (server to client communication). So far our focus is mainly on optimizing for upload efficiency as uploads are generally slower. Let us modify the JSON config above to add a *channel* key to specify channel subconfigs. This controls selection of the compression method, as well as metrics tracking. When *report_communication_metrics* is enabled, the mean upload and download bytes (averaged across participating clients in a round) are reported. Let us consider the simplest compression supported in the simulator, namely the [half-precision channel](https://github.com/facebookresearch/FLSim/blob/main/channels/half_precision_channel.py). Here, client updates (which by default are 32-bit floating points) are compressed to 16-bit. We can specify the compression method with the *_base_* key here in the sub-config.

In [44]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_half_precision_channel"
}
cfg = fl_config_from_json(json_config)


/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."


After defining the JSON config and converting to an FL config, we instantiate the trainer and start training with half-precision compression. While the server sends full-precision models to clients, the client uploads consume half the number of bytes as expected.

In [45]:
run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.3368776977062224
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 13.68
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:20,  1.09s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.1055475905537606
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 24.68
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.1054040387272837
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 23.28
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:19,  1.09s/round]

...

Round:  90%|█████████ | 18/20 [00:19<00:02,  1.09s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.355839768052101
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 51.28
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.258171808719635
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 55.08
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:20<00:01,  1.09s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.26211207062006
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 55.2
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.2380948312580586
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 56.96
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.2786135971546173
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 53.52
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, glo

Epoch:   0%|          | 0/1 [00:23<?, ?epoch/s]


We now look at the scalar quantization channel, which quantizes tensors in each client update to between 1 and 8 bits. The channel supports both *per-tensor quantization* and *per-channel quantization*. Let us consider the former which is the simplest case, where each tensor shares the same zero-point and scale parameters. The channel configuration can be now modified to select this channel, with its associated per-tensor quantization flag and number of bits (*n_bits*) parameter.

In [46]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_scalar_quantization_channel",
  "n_bits": 3,
  "quantize_per_tensor": True
}

run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.3375110030174255
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 14.6
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:20,  1.10s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.3128815978765487
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 14.64
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.1466801568865774
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 21.52
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:19,  1.11s/round]

...

Round:  90%|█████████ | 18/20 [00:19<00:02,  1.11s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.2921988151967525
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 53.04
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.3031659096479415
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 54.52
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:21<00:01,  1.11s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.349413202702999
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 52.04
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.2935495600104332
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 53.08
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.3623090006411076
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 52.32
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, g

Epoch:   0%|          | 0/1 [00:23<?, ?epoch/s]


Note how the number of bytes during upload is not exactly 11847 (=126376 * 3/32). The additional overhead of 793 bytes is due to the upload messages also including scale and zero-point information, which is necessary server-side to dequantize the client update tensors. We now wrap up this tutorial by looking at another channel, [sparse mask channel](https://github.com/facebookresearch/FLSim/blob/main/channels/sparse_mask_channel.py) which implements sparsity on the client updates. The sparsity mask is client-specific, and the simulator supports two approaches to determine this mask : *random* and *Top-K*. Random mask is self-explanatory, with the per-tensor sparsity mask being set randomly based on the sparsity level, and *Top-K* sparsity masking out the lowest magnitude elements within each model update tensor. Let us now define a config using the Top-K mask with a sparsity level of 99%, and run a round of training using this compression. 

In [47]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_sparse_mask_channel",
  "proportion_of_zero_weights":0.99,
  "sparsity_method": "topk"
}

run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.3019975513219832
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 13.84
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:21,  1.11s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.362009060382843
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 12.12
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 5245.25
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.2777017533779143
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 14.6
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:20,  1.12s/round]

...

Round:  90%|█████████ | 18/20 [00:20<00:02,  1.12s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.630456741154194
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 40.52
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 5245.25
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.6380363792181014
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 41.08
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:21<00:01,  1.11s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.5991919979453086
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 42.72
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 5245.25
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.6388011947274208
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 40.4
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.5713405102491378
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 43.64
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 5245.25
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, g

Epoch:   0%|          | 0/1 [00:23<?, ?epoch/s]


While the non-sparse parameters consume only 1263 bytes, we also need to pack in additional information about the sparsity pattern per tensor. The simulator supports two ways to do this: *bitmask* and *coo*. In the bitmask setting, the sparsity pattern is transported as a concatenated bitstream. We also support another mode *coo*, which follows the popular COO format for encoding sparse tensors as described in [torch.sparse](https://pytorch.org/docs/stable/sparse.html#sparse-coo-tensors) documentation. Here, the non-sparse tensor weights are transmitted along with their 64-bit integer coordinates. Generally *bitmask* has a lower overhead than *coo* in terms of bytes transmitted. Let us conclude our tutorial by running a round of Cifar-10 training with *coo* bytes measurement.

In [48]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_sparse_mask_channel",
  "proportion_of_zero_weights": 0.99,
  "sparsity_method": "topk",
  "compressed_size_measurement": "coo"
}

run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.301069360971451
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 14.72
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:21,  1.14s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.3707367897033693
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 14.04
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 11032.0
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.2785249024629595
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 15.04
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:20,  1.13s/round]

...

Round:  90%|█████████ | 18/20 [00:20<00:02,  1.16s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.5557768046855927
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 44.44
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 11032.0
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.5778911039233208
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 42.56
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:21<00:01,  1.14s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.522291925549507
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 44.84
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 11032.0
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.5901262491941452
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 42.08
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.5773373901844026
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 43.36
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 11032.0
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, g

Epoch:   0%|          | 0/1 [00:23<?, ?epoch/s]


This tutorial has provided a concise overview of how to use the existing channel features in FLSim for communication efficiency. This should be useful for any development work on experimenting with different compression approaches to enable scalable federated learning. Please refer to https://github.com/facebookresearch/FLSim/tree/main/channels for the code pointers to implementation of different channels as well as the communication metrics reporting.